In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install konlpy
!pip install kss

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder; LE = LabelEncoder()
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Mecab

if __name__ == "__main__":
    max_len = 32

    mecab = Mecab()

    model = Word2Vec.load('/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/word2vec')

    depression_dataset_path = r'/gdrive/My Drive/한양대학교/2020 프로젝트 학기제/TEST/depression_dataset_맞춤법O.csv'

    depression_dataset = pd.read_csv(depression_dataset_path, encoding='cp949')

    df = []

    for i in range(len(depression_dataset)):
        df.append(mecab.morphs(depression_dataset.iloc[i,0]))

    for i in range(len(df)):
        for j in range(len(df[i])):
            try:
                df[i][j] = model.wv.get_vector(df[i][j]).tolist()
            except:
                df[i][j] = False

    for i in range(len(df)):
        while False in df[i]:
            df[i].remove(False)

    for i in range(len(df)):
        if len(df[i]) >= max_len:
            df[i] = df[i][-32:]
        else:
            for j in range(max_len-len(df[i])):
                df[i].insert(0,0)

    for i in range(len(df)):
        for j in range(len(df[i])):
            if df[i][j] == 0:
                df[i][j] = [0]*100

    depression_dataset['label_1'] = LE.fit_transform(depression_dataset['label_1'])

    x_train, x_test, y_train, y_test = train_test_split(df, depression_dataset['label_1'], test_size=0.1, random_state=42)

    x_train = np.array(x_train).reshape((len(x_train), max_len*100))
    x_test = np.array(x_test).reshape((len(x_test), max_len*100))

    print('train_shape : {} / {}'.format(x_train.shape, y_train.shape))
    print('test_shape : {} / {}'.format(x_test.shape, y_test.shape))

    model = SVC(kernel='linear', random_state=42)
    # model = SVC(kernel='poly', random_state=42)
    # model = SVC(kernel='rbf', random_state=42)
    # model = SVC(kernel='sigmoid', random_state=42)

    model.fit(x_train, y_train)

    y_test = list(y_test)
    y_pred = list(model.predict(x_test))

    print('accuracy_score = ', accuracy_score(y_test, y_pred))

    print(classification_report(y_test, y_pred))
    print(pd.crosstab(pd.Series(y_test), pd.Series(y_pred), rownames=['True'], colnames=['Predicted']))